# Predicting Game Newsletter Subscriptions Using Player Playtime and Experience Level Data

# 1. Background Information

A research video game server created by Frank Wood and his research team at UBC aimed to collect data both in game about participant gameplay sessions, as well as external participant traits. A survey was taken by research participants prior to their playing sessions and this data has been collected into the players.csv file. 

# 2. Questions

#### ***Broad question:*** 
=What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

#### ***Specific question:*** 
Can experience level and play time predict subscription in the players.csv dataset? 



The dataset 'players.csv' describes 196 unique players, including data about each player.
Numbers of observations: 196,
numbers of variables: 7

|Variable     |   Type    |     Description  |
|-------------|-----------|------------------------------|
|experience   | Character |Player's experience level|
|hashedEmail  | Character |Player's ID|
|name         | Character |  Player's name               |
|gender       | Character  | Player's gender|
|played_hours | Double      |Player's playtime (in hours)|
|Age           |Double     | Player's age|
|subscribe     |Logical     |Whether the player subscribed to the newsletter|
